In [1]:
import numpy as np
from scipy.optimize import linprog

Допустим, у нас есть n товаров с заданными стоимостями vi и массой wi. В сумку убирается с кг. Сколько какого товара взять, чтобы сумма всех стоимостей товаров была наибольшей?

In [2]:
values = [4, 2, 1, 7, 3, 6]
weights = [5, 9, 8, 2, 6, 5]
C = 15
n = 6

Сформулируем задачу линейного программирования:

In [3]:
c = -np.array(values)
c

array([-4, -2, -1, -7, -3, -6])

In [4]:
A = np.array(weights).reshape(1, 6)
A

array([[5, 9, 8, 2, 6, 5]])

In [5]:
b = np.array(C).reshape(1,)
b

array([15])

Передаем определенные параметры задачи ЛП в функцию linprog

In [6]:
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.500000000030745
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24886776e-11])
  status: 0
 success: True
       x: array([6.18738521e-14, 1.05853304e-12, 1.21475941e-13, 7.50000000e+00,
       4.00246685e-13, 4.71394154e-13])

Предположим, что товары в задаче нельзя дробить и решим задачу целочисленного линейного программирования.

Scipy этого делать не умеет. Будем использовать новую библиотеку **cvxpy**.

In [18]:
linprog?

In [7]:
!pip install --user cvxpy

In [8]:
import cvxpy

In [9]:
values = [4, 2, 1, 7, 3, 6]
weights = [5, 9, 8, 2, 6, 5]
C = 15
n = 6

c = - np.array(values)
A = np.array(weights)         #shape = (6,)
A = np.expand_dims(A, 0)      #shape = (1,6)
b = np.array([C])

Определяем искомые переменные в количестве n=6 и с условием, что они целочисленные

In [10]:
x = cvxpy.Variable(shape=n, integer=True)

Задаем условия в явном виде:

In [11]:
constraint = (A @ x <= b)
total_value = (c @ x)

In [12]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [13]:
problem.solve(solver='ECOS_BB')

-138412039.0000002

Теперь положительные $x$

In [14]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c @ x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve(solver='ECOS_BB')

/notebooks/home/.local/lib/python3.6/site-packages/cvxpy/problems/problem.py:1126: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


-49.000000015906025

In [15]:
x.value

array([7.01265807e-10, 7.99333027e-10, 3.58703130e-10, 7.00000000e+00,
       4.67143021e-10, 9.34955115e-10])

Теперь $x = 0$ или $1$

In [16]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve(solver='ECOS_BB')

/notebooks/home/.local/lib/python3.6/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


-17.00000000382157

In [17]:
x.value

array([1.00000000e+00, 2.25474413e-10, 2.07396078e-10, 1.00000000e+00,
       2.24003299e-10, 1.00000000e+00])

Составьте оптимальный план перевозок, со Склада № 1 и Склада № 2, в три торговых центра, с учётом тарифов, запасов и потребностей, которые указаны в таблице:

                    | ТЦ1 (110 шт) | ТЦ2 (150 шт) | ТЦ3 (140 шт)
    Склад1 (180 шт) |   2 y.e.     |   5 y.e.     |   3 y.e.
    Склад2 (220 шт) |   7 y.e.     |   7 y.e.     |   6 y.e.

In [24]:
n = 6

c = np.array([2, 5, 3, 7, 7, 6])
A_eq = np.array([[1, 0, 0, 1, 0, 0], 
                 [0, 1, 0, 0, 1, 0],
                 [0, 0, 1, 0, 0, 1]])
b_eq = np.array([110, 150, 140])

A_ub = np.array([[1, 1, 1, 0, 0, 0], 
                 [0, 0, 0, 1, 1, 1]])
b_ub = np.array([180, 220])

Решим с помощью scipy.optimize.linprog

In [26]:
linprog(c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq)

     con: array([4.39231886e-06, 6.01910361e-06, 5.61240731e-06])
     fun: 1899.9999256826495
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([7.20364935e-06, 8.82018043e-06])
  status: 0
 success: True
       x: array([1.09999995e+02, 4.43103329e-08, 6.99999980e+01, 8.98577274e-07,
       1.49999994e+02, 6.99999963e+01])

Решим с помощью cvxpy

In [29]:
x = cvxpy.Variable(shape=n, integer=True)

constraint_ub = (A_ub @ x <= b_ub)
constraint_eq = (A_eq @ x == b_eq)
x_positive = (x >= 0)

total_value = c @ x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint_ub, constraint_eq, x_positive])
problem.solve(solver='ECOS_BB')

1900.0000000102355

Эксперт курса предлагает решать так (по факту то же самое, но оформление более понятно):

У меня y - искомая переменная, c - матрица затрат, total_value - это то, что минимизируется в задаче - сумма элементов матрицы поэлементного умножения y и c
Ограничения:
 - первые два - суммы по рядам (ограничения, связанные с объёмом продукции на складах)
 - следующие три - суммы по столбцам (ограничения, связанные с объёмом необходимой продукции в ТЦ)
 - последнее - неотрицательность элементов y

In [31]:
c = np.array([[2, 5, 3], [7, 7, 6]])
y = cvxpy.Variable(shape=(2, 3), integer=True)
constraint = [cvxpy.sum(y[0, :]) <= 180, 
              cvxpy.sum(y[1, :]) <= 220, 
              cvxpy.sum(y[:, 0]) == 110, 
              cvxpy.sum(y[:, 1]) == 150, 
              cvxpy.sum(y[:, 2]) == 140, 
              y >= 0]
total_value = cvxpy.sum(cvxpy.multiply(c, y))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraint)
problem.solve(solver='ECOS_BB')

1900.0000000102355

Решите задачу о назначениях:

Задачи/Исполнители

      | 1    | 2    | 3    | 4    | 5
    1 | 1000 | 12   | 10   | 19   | 9
    2 | 12   | 1000 | 3    | 7    | 2 
    3 | 10   | 3    | 1000 | 6    | 20 
    4 | 19   | 7    | 6    | 1000 | 4
    5 | 9    | 2    | 20   | 4    | 1000

In [46]:
c = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

y = cvxpy.Variable(shape=(5, 5), boolean=True)
constraints = [cvxpy.sum(y[0, :]) == 1, 
              cvxpy.sum(y[1, :]) == 1, 
              cvxpy.sum(y[2, :]) == 1,
              cvxpy.sum(y[3, :]) == 1,
              cvxpy.sum(y[4, :]) == 1,              
              cvxpy.sum(y[:, 0]) == 1, 
              cvxpy.sum(y[:, 1]) == 1, 
              cvxpy.sum(y[:, 2]) == 1, 
              cvxpy.sum(y[:, 3]) == 1, 
              cvxpy.sum(y[:, 4]) == 1]

total_value = cvxpy.sum(cvxpy.multiply(c, y))

In [47]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraints)
problem.solve(solver='ECOS_BB')

31.99999999996171

Условия можно записать более компактно:

In [48]:
c = np.array([[1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2], 
    [10, 3, 1000, 6, 20], 
    [19, 7, 6, 1000, 4], 
    [8, 2, 20, 4, 1000]])
y = cvxpy.Variable(shape=(5,5), boolean=True)
constraints = [
    cvxpy.sum(y, axis=0) == np.ones(5),
    cvxpy.sum(y, axis=1) == np.ones(5)
]
total_value = cvxpy.sum(cvxpy.multiply(y, c))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraints)
problem.solve(solver='ECOS_BB')

31.999999999961364

Решить задачу Коммивояжера

In [57]:
from itertools import product

In [56]:
# коэффициенты стоимости пути между точками
c = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

In [58]:
y = cvxpy.Variable(shape=(5,5), boolean=True)

# Условия, в каждую точку надо зайти 1 раз и выйти 1 раз
constraints = [
    cvxpy.sum(y, axis=0) == np.ones(5),
    cvxpy.sum(y, axis=1) == np.ones(5)
]

In [59]:
# Добавим условие исключения подциклов 
u = cvxpy.Variable(shape=5, boolean=True)

for i,j in product(range(5), range(5)):
    if i >= 0 and j >= 1 and i != j:
        u[i] - u[j] + 5*y[i, j] <= 4

In [60]:
total_value = cvxpy.sum(cvxpy.multiply(y, c))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraints)
problem.solve(solver='ECOS_BB')

31.999999999961364